In [1]:
import xarray as xr
import plotly.graph_objects as go
import pandas as pd
import cf_xarray.units  # noqa: F401
import pint_xarray  # noqa: F401
from datetime import datetime
from pathlib import Path

pd.options.plotting.backend = "plotly"

In [2]:
station_name = "BATS"
latitude = 31.6
longitude = -64.2
release_folder = "../../../../release/"

---

## Load datasets


In [3]:
data_bats = xr.load_dataset("../../3_post_processed/bats_product.nc")
data_bats

<xarray.Dataset> Size: 16MB
Dimensions:                        (time: 4428, is_day: 2, depth: 8,
                                    latitude: 1, longitude: 1, sieve_size: 5)
Coordinates:
  * time                           (time) datetime64[ns] 35kB 1988-10-21T04:5...
  * is_day                         (is_day) bool 2B False True
  * depth                          (depth) float64 64B 50.0 100.0 ... 400.0
  * latitude                       (latitude) float64 8B 31.6
  * longitude                      (longitude) float64 8B -64.2
  * sieve_size                     (sieve_size) float64 40B 0.2 0.5 1.0 2.0 5.0
Data variables: (12/37)
    dry_weight_vol_water_ratio     (time, is_day, depth, latitude, longitude, sieve_size) float64 3MB ...
    wet_weight_vol_water_ratio     (time, is_day, depth, latitude, longitude, sieve_size) float64 3MB ...
    pres                           (time, depth, latitude, longitude) float64 283kB ...
    temp                           (time, depth, latitude, longitude) float64 283kB ...
    salt                           (time, depth, latitude, longitude) float64 283kB ...
    lt1                            (time, depth, latitude, longitude) float64 283kB ...
    ...                             ...
    particulate_biogenic_silica    (time, depth, latitude, longitude) float64 283kB ...
    particulate_lithogenic_silica  (time, depth, latitude, longitude) float64 283kB ...
    prochlorococcus                (time, depth, latitude, longitude) float64 283kB ...
    synechococcus                  (time, depth, latitude, longitude) float64 283kB ...
    picoeukaryotes                 (time, depth, latitude, longitude) float64 283kB ...
    nanoeukaryotes                 (time, depth, latitude, longitude) float64 283kB ...

---

## Vizualization

### BAT station


In [4]:
zoo_bats = (
    data_bats["dry_weight_vol_water_ratio"]
    .dropna("time", how="all")
    .mean("depth")
    .isel(latitude=0, longitude=0)
    .sum("sieve_size")
)
bats_units = data_bats["dry_weight_vol_water_ratio"].pint.quantify().pint.units
zoo_bats_d_n = zoo_bats.to_dataframe().pivot_table(
    index=["time"], columns=["is_day"], values=["dry_weight_vol_water_ratio"]
)
zoo_bats_d_n.columns = ["night", "day"]
# transform 0 to nan
zoo_bats_d_n = zoo_bats_d_n.map(lambda x: None if x == 0 else x)

In [5]:
day = zoo_bats_d_n["day"].dropna()
night = zoo_bats_d_n["night"].dropna()
go.Figure(
    data=[
        go.Scatter(x=day.index, y=day, mode="lines", name="Day"),
        go.Scatter(x=night.index, y=night, mode="lines", name="Night"),
    ]
).update_layout(
    title="BATS : zooplankton dry weight by cube meter of sea water",
    xaxis_title="Time",
    yaxis_title="Dry Weight (mg/m3)",
    legend_title="Time of Day",
).show()

---

## Export the products in netCDF format

First, write the function to prepare the forcing and save it as NetCDF file.


In [6]:
as_xarray = xr.Dataset.from_dataframe(zoo_bats_d_n)
as_xarray = as_xarray.expand_dims(
    {"latitude": [latitude], "longitude": [longitude], "layer": [0]}
)
as_xarray["time"].attrs = {
    "axis": "T",
    "standard_name": "time",
    "long_name": "time",
}
as_xarray["night"].attrs = {
    "units": "mg/m3",
    "description": "Zooplankton dry weight by cube meter of sea water at night",
}
as_xarray["day"].attrs = {
    "units": "mg/m3",
    "description": "Zooplankton dry weight by cube meter of sea water during the day",
}
as_xarray.attrs = {
    "station": station_name,
    "creation date": datetime.now().isoformat(),
}
as_xarray["layer"].attrs = {
    "flag_values": [0, 1, 2],
    "flag_meanings": "epipelagic, upper-mesopelagic, lower-mesopelagic",
    "desc": "The layer depth according to Seapodym definition",
    "standard_name": "layer",
    "long_name": "Layer depth",
    "axis": "Z",
}
as_xarray["latitude"].attrs = {
    "standard_name": "latitude",
    "axis": "Y",
    "units": "degrees_north",
    "long_name": "Latitude",
}
as_xarray["longitude"].attrs = {
    "standard_name": "longitude",
    "axis": "X",
    "units": "degrees_east",
    "long_name": "Longitude",
}
as_xarray = as_xarray.transpose("time", "latitude", "longitude", "layer")
as_xarray = as_xarray.chunk({"time": -1})
as_xarray.to_zarr(Path(release_folder) / f"{station_name}_zooplankton.zarr", mode="w")